# 개요

# Prepare

# 1. 초기 세팅

## Library & data load

In [232]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [233]:
# 처리용 df와 오리지널 버전 남겨놓음
df = pd.read_csv("/Users/luci031/Downloads/auction/data_origin/auction_202101.csv",encoding='euc_kr')
org_df = pd.read_csv("/Users/luci031/Downloads/auction/data_origin/auction_202101.csv",encoding='euc_kr')

# Pre-Processing

## Basic info

In [234]:
df.head()

,품목명,단위,등급,가격,산지,친환경구분,입력일
0,[가리비국산]가리비국산,5kg,보통(3등),24000,경남 남해,일반,20210104
1,[가리비국산]가리비국산,5kg,보통(3등),22000,"경남 마산(고성,진해)",일반,20210104
2,[가리비국산]가리비국산,5kg,보통(3등),23000,경남 남해,일반,20210104
3,[가리비국산]가리비국산,5kg,보통(3등),25000,"경남 마산(고성,진해)",일반,20210104
4,[가리비국산]가리비국산,5kg,보통(3등),23500,"경남 마산(고성,진해)",일반,20210104


In [235]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 409189 entries, 0 to 409188
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   품목명     409189 non-null  object
 1   단위      409189 non-null  object
 2   등급      409189 non-null  object
 3   가격      409189 non-null  int64 
 4   산지      405456 non-null  object
 5   친환경구분   409189 non-null  object
 6   입력일     409189 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 21.9+ MB


In [236]:
# 산지 제외하고는 결측치 없는 것으로 확인
df.isnull().sum()

품목명         0
단위          0
등급          0
가격          0
산지       3733
친환경구분       0
입력일         0
dtype: int64

- 결측치 처리

In [237]:
###
# 양이 많지 않으므로 결측치 제거
df = df.dropna(axis=0)

- 영문명으로 전환

In [238]:
df = df.rename(columns = {'품목명':'prd',
                          '단위':'scale','등급':'class',
                          '가격':'price',
                          '산지':'origin',
                          '친환경구분':'eco',
                          '입력일':'reg_date'})

## column 별 정보

### class

In [239]:
# 이름 모두 영어로 변경
# 9등과 없음 모두 9로 변경
df['class'].value_counts().index

Index(['특(1등)', '상(2등)', '보통(3등)', '9등(등외)', '없음', '4등', '5등', '6등', '7등',
       '8등'],
      dtype='object')

In [240]:
df111 = pd.DataFrame([['특(1등)', 1], ['상(2등)',2], ['보통(3등)',3], ['9등(등외)',9], ['없음',9], ['5등',5], ['4등',4], ['6등',6], ['7등',7],['8등',8]],columns=['class','new_class'])

In [241]:
df2 = df.merge(df111)
df2 = df2.drop(columns='class')

In [242]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[가리비국산]가리비국산,5kg,24000,경남 남해,일반,20210104,3
1,[가리비국산]가리비국산,5kg,22000,"경남 마산(고성,진해)",일반,20210104,3
2,[가리비국산]가리비국산,5kg,23000,경남 남해,일반,20210104,3
3,[가리비국산]가리비국산,5kg,25000,"경남 마산(고성,진해)",일반,20210104,3
4,[가리비국산]가리비국산,5kg,23500,"경남 마산(고성,진해)",일반,20210104,3
...,...,...,...,...,...,...,...
405451,[토마토]대저토마토,2.5kg,4000,부산 강서구,일반,20210130,7
405452,[토마토]대저토마토,5kg,7000,경남 김해시,일반,20210130,7
405453,[토마토]대저토마토,5kg,14000,경남 김해시,일반,20210130,7
405454,[토마토]토마토(기타),2.5kg,12000,부산 강서구,일반,20210130,7


### eco

In [243]:
# 영문명으로 전환
# 전환기 -> 유기농 재배로 전환중
df2.loc[df2["eco"]=="일반","eco"] = "normal"
df2.loc[df2["eco"]=="우수농산물","eco"] = "good_prd"
df2.loc[df2["eco"]=="무농약","eco"] = "ecofriendly"
df2.loc[df2["eco"]=="유기농","eco"] = "organic"
df2.loc[df2["eco"]=="품질인증","eco"] = "certified"
df2.loc[df2["eco"]=="전환기","eco"] = "ing"

In [244]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[가리비국산]가리비국산,5kg,24000,경남 남해,normal,20210104,3
1,[가리비국산]가리비국산,5kg,22000,"경남 마산(고성,진해)",normal,20210104,3
2,[가리비국산]가리비국산,5kg,23000,경남 남해,normal,20210104,3
3,[가리비국산]가리비국산,5kg,25000,"경남 마산(고성,진해)",normal,20210104,3
4,[가리비국산]가리비국산,5kg,23500,"경남 마산(고성,진해)",normal,20210104,3
...,...,...,...,...,...,...,...
405451,[토마토]대저토마토,2.5kg,4000,부산 강서구,normal,20210130,7
405452,[토마토]대저토마토,5kg,7000,경남 김해시,normal,20210130,7
405453,[토마토]대저토마토,5kg,14000,경남 김해시,normal,20210130,7
405454,[토마토]토마토(기타),2.5kg,12000,부산 강서구,normal,20210130,7


### reg_date

In [245]:
df2['reg_date'] = df2['reg_date'].astype('str')

In [246]:
df2['reg_date'] = pd.to_datetime(df2['reg_date'].astype('str'),format='%Y%m%d')

In [247]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[가리비국산]가리비국산,5kg,24000,경남 남해,normal,2021-01-04,3
1,[가리비국산]가리비국산,5kg,22000,"경남 마산(고성,진해)",normal,2021-01-04,3
2,[가리비국산]가리비국산,5kg,23000,경남 남해,normal,2021-01-04,3
3,[가리비국산]가리비국산,5kg,25000,"경남 마산(고성,진해)",normal,2021-01-04,3
4,[가리비국산]가리비국산,5kg,23500,"경남 마산(고성,진해)",normal,2021-01-04,3
...,...,...,...,...,...,...,...
405451,[토마토]대저토마토,2.5kg,4000,부산 강서구,normal,2021-01-30,7
405452,[토마토]대저토마토,5kg,7000,경남 김해시,normal,2021-01-30,7
405453,[토마토]대저토마토,5kg,14000,경남 김해시,normal,2021-01-30,7
405454,[토마토]토마토(기타),2.5kg,12000,부산 강서구,normal,2021-01-30,7


### price

In [248]:
# 계산을 위해 scale 값 미리 변경
# 현재 값들은 string이므로 float 형태로 변환할 필요가 있음
# 몇몇 이상치 처리도 진행
def scale_checker(scale):
  if scale.startswith('.')==True:
    return format(float('0'+scale[:-2]),'.2f')
  else:
    return format(float((scale[:-2])),'.2f')
df2['scale'] = df2['scale'].apply(scale_checker)
df2['scale'] = df2['scale'].astype('float')
# df2['scale'] = format(round(df2['scale'].apply(pd.to_numeric),2),'.2f')

In [249]:
df2['scale'].unique()

array([    5.  ,     1.  ,     3.  ,     1.5 ,     4.  ,     4.5 ,
           8.  ,     5.01,    10.  ,    20.  ,    15.  ,     5.4 ,
           6.  ,     2.  ,     0.05,     0.15,     0.2 ,     2.7 ,
           0.8 ,     0.9 ,     1.8 ,     1.56,     1.7 ,     1.9 ,
           1.6 ,     0.7 ,     9.5 ,     5.1 ,     0.5 ,     3.6 ,
           3.4 ,    21.5 ,    23.  ,     7.  ,    13.  ,     9.  ,
           1.4 ,    12.  ,     3.2 ,    18.  ,     2.5 ,    11.  ,
           4.4 ,     3.5 ,     8.01,     2.2 ,     4.2 ,     3.9 ,
           3.7 ,     7.5 ,     2.1 ,    10.2 ,     2.4 ,    11.5 ,
          16.5 ,    16.  ,     2.3 ,     3.8 ,     6.6 ,     1.3 ,
         101.  ,     8.5 ,    12.5 ,    17.  ,     2.8 ,     3.3 ,
          22.  ,     4.3 ,    30.  ,    21.  ,    22.5 ,    14.  ,
          21.8 ,     5.45,    13.5 ,    19.  ,    40.  ,    10.3 ,
           1.1 ,     6.5 ,    26.  ,     6.4 ,     9.8 ,     2.9 ,
          50.  ,     4.1 ,     4.8 ,     4.6 ,     4.7 ,     9

In [250]:
# 너무 거래값이 작은 값들은 삭제
pd.options.display.float_format = '{:.5f}'.format
df2['price'].describe()

count     405456.00000
mean       28125.19973
std       104118.25583
min            0.00000
25%         9500.00000
50%        18600.00000
75%        33000.00000
max     50028200.00000
Name: price, dtype: float64

In [251]:
df2 = df2.drop(df2[df2['price']<500].index)

In [252]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[가리비국산]가리비국산,5.00000,24000,경남 남해,normal,2021-01-04,3
1,[가리비국산]가리비국산,5.00000,22000,"경남 마산(고성,진해)",normal,2021-01-04,3
2,[가리비국산]가리비국산,5.00000,23000,경남 남해,normal,2021-01-04,3
3,[가리비국산]가리비국산,5.00000,25000,"경남 마산(고성,진해)",normal,2021-01-04,3
4,[가리비국산]가리비국산,5.00000,23500,"경남 마산(고성,진해)",normal,2021-01-04,3
...,...,...,...,...,...,...,...
405451,[토마토]대저토마토,2.50000,4000,부산 강서구,normal,2021-01-30,7
405452,[토마토]대저토마토,5.00000,7000,경남 김해시,normal,2021-01-30,7
405453,[토마토]대저토마토,5.00000,14000,경남 김해시,normal,2021-01-30,7
405454,[토마토]토마토(기타),2.50000,12000,부산 강서구,normal,2021-01-30,7


In [253]:
# kg 당 가격 생성

df2['price_kg'] = round((df2['price']/df2['scale']),2)

### prd
- 품목명 정리
- 품목 종류 정리

In [254]:
# 대부분 [상품 종류]상품명 으로 정리되어 있는걸 알 수 있음
df2['prd'].value_counts()

[딸기]설향          13932
[오이]백다다기         8176
[딸기]설향(딸기)       7845
[깻잎]깻잎           5951
[시금치]시금치         5733
                ...  
[아욱]아욱(아욱)          1
[깻잎]깻순(깻순)          1
[대구포수입]대구포수입        1
[깐소라수입]깐소라수입        1
[포도]포도(세레단)         1
Name: prd, Length: 1175, dtype: int64

In [255]:
# 정리를 위해 상품명 삭제
def prd_sort(words):
  p = re.compile('[ㄱ-힣]+')
  word = re.search(p,words)
  return word.group()

df2['prd'] = df2['prd'].apply(prd_sort)

In [256]:
# 기타 품목 삭제
df2 = df2.drop(df2[df2['prd']=='기타'].index)

In [257]:
# 국산,수입산,깐,기타 삭제
def prd_sort2(words):
  p = re.compile('[^국산수입깐기타]+')
  word = re.search(p,str(words))
  return word.group()

df2["prd"] = df2["prd"].apply(prd_sort2)


In [296]:
# 몇몇 품목들 정리가 필요
a = sorted(df2['prd'].unique())
for i in range(len(a)):
  if i%15 != 0:
    print(a[i],end=' ')
  else:
    print(a[i],end=' ')
    print()

가리비 
가물치 가오리 가오리채 가자미 가지 갈치 감 감귤 감말랭이 감자 갑오징어 갓 개량조개 개조개 건고추 
게 겨자 고구마 고둥 고들빼기 고등어 고사리 고수 고추 고추잎 곤달비 곶감 과채류 굴 귤 
근대 근채류 금감 김 깻잎 꼴뚜기 꽁치 꽈리고추 낙지 냉이 노루궁뎅이버섯 논우렁이 눈볼대 느타리버섯 다래 
단감 달래 당귀 당근 대구 대추 대파 더덕 도다리 도라지 돌나물 돔 동자개 동죽 두릅 
두리안 딸기 땅콩 떫은감 라임 람부탄 레몬 리치 마 마늘 만가닥버섯 만감 망고 매생이 머위대 
멍게 메기 메론 명태 모과 모시조개 모자반 무 무순 무청 문어바지락 미꾸라지 미나리 미더덕 미역 
민물돔 민물새우 민어 바나나 바다가재 바지락 박 밤 방울토마토 방풍나물 배 배추 백새우 백합 뱀장어 
버섯 버섯류 병어 봄동배추 부세 부지갱이 부추 북방대합조개 붕어 붕장어 브로콜리 블루베리 비타민 비트 빙어 
빨간양배추 사과 삼치 상추 새꼬막 새송이버섯  새싹 새우 새조개 샐러리 생강 석류 소라 속새 수삼 
숙주나물 숫꽃게 시금치 쌈채류 쑥 쑥갓 씀바귀 아귀 아보카도 아스파라거스 아욱 알배기 알타리무 앵두 야자 
야콘 양배추 양상추 양송이 양채류 양파 어류 얼갈이배추 연근 연어 열무 엽경채류 오렌지 오이 오징어 
완두콩 용과 우럭조개 우뭇가사리 우엉 유채 임연수어 잉어 자라 자몽 장어 재첩 적겨자 적근대 적새우 
적양배추 전복 조개 조기 죽순 중하 쪽파 쭈꾸미 참꼬막 참나물 참다래 참당귀 참외 참조기 청경채 
청어 체리 총각무 취나물 치커리 컬리플라워 케일 코코넛 콜라비 콩 콩나물 키위 키조개 토마토 톳 
파 파래김 파슬리 파인애플 파프리카 팽이버섯 포도 표고버섯 풋고추 피망 피조개 해삼 해파리 향어 호박 
홍고추 홍어 홍합 

In [259]:
### 예시
# df2[(df2['prd']=='유채') & (df2['price']==4500)]['prd'].unique()
# df2[df2['prd']=='유채']['prd'].unique()

In [260]:
### 원래 단어 뭐였는지 검색
# org_df[org_df['품목명'].str.contains('느')]['품목명'].unique()

In [261]:
### 현재 이름 검색
# df2[df2['prd']=='느']

In [262]:
###
# JG

# 고 -> 고수로 변경
df2.loc[df2["prd"]=="고","prd"] = "고수"
# 과실류, 과일류, 과채, 채소 -> 과채류로 통합
df2.loc[df2["prd"]=="과실류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일","prd"] = "과채류"
df2.loc[df2["prd"]=="과채","prd"] = "과채류"
df2.loc[df2["prd"]=="채소","prd"] = "과채류"
# 고들빼 -> 고들빼기로 변경
df2.loc[df2["prd"]=="고들빼","prd"] = "고들빼기"
# 꼴뚜 -> 꼴두기로 변경
df2.loc[df2["prd"]=="꼴뚜","prd"] = "꼴뚜기"
# 그레이프푸룻 -> 자몽으로 통합
df2.loc[df2["prd"]=="그레이프푸룻","prd"] = "자몽"

# 느 -> 느타리버섯으로 통합
df2.loc[df2["prd"]=="느","prd"] = "느타리버섯"

# 대구 관련 품목들 정리
df2.loc[df2["prd"]=="대구고니","prd"] = "대구"
df2.loc[df2["prd"]=="대구머리","prd"] = "대구"
df2.loc[df2["prd"]=="대구포","prd"] = "대구"
# 듀리안 -> 두리안으로 통합
df2.loc[df2["prd"]=="듀리안","prd"] = "두리안"
# 딸 -> 딸기로 변경
df2.loc[df2["prd"]=="딸","prd"] = "딸기"

# 만가닥 -> 만가닥버섯으로 변경
df2.loc[df2["prd"]=="만가닥","prd"] = "만가닥버섯"
# 맛 -> 맛조개로 변경
df2.loc[df2["prd"]=="맛","prd"] = "맛조개"
# 망고스턴 -> 망고스틴으로 수정
df2.loc[df2["prd"]=="망고스턴","prd"] = "망고스틴"
# 메 -> 메기로 변경
df2.loc[df2["prd"]=="메","prd"] = "메기"
# 명태 관련 품목들 정리
df2.loc[df2["prd"]=="명태곤니","prd"] = "명태"
df2.loc[df2["prd"]=="명태알","prd"] = "명태"
df2.loc[df2["prd"]=="명태포","prd"] = "명태"
# 무우 -> 무로 통합
df2.loc[df2["prd"]=="무우","prd"] = "무"

# 방풍 -> 방풍나물로 통합
df2.loc[df2["prd"]=="방풍","prd"] = "방풍나물"
# 버섯류 -> 버섯으로 변경
df2.loc[df2["prd"]=="버섯","prd"] = "버섯"
# 벗,버찌 -> 체리로 통합
df2.loc[df2["prd"]=="벗","prd"] = "체리"
df2.loc[df2["prd"]=="버찌","prd"] = "체리"
# 브로커리, 브로코리 -> 브로콜리로 통합
df2.loc[df2["prd"]=="브로커리","prd"] = "브로콜리"
df2.loc[df2["prd"]=="브로코리","prd"] = "브로콜리"

# 삼 -> 수삼으로 변경
df2.loc[df2["prd"]=="삼","prd"] = "수삼"
# 새우살 -> 새우로 변경
df2.loc[df2["prd"]=="새우살","prd"] = "새우"
# 세러리, 셀러리 -> 샐러리로 통합
df2.loc[df2["prd"]=="세러리","prd"] = "샐러리"
df2.loc[df2["prd"]=="셀러리","prd"] = "샐러리"
# 생고추 -> 고추로 통합
df2.loc[df2["prd"]=="생고추","prd"] = "고추"

# 아보카드 -> 아보카도로 변경
df2.loc[df2["prd"]=="아보카드","prd"] = "아보카도"
# 아스파라가스 -> 아스파라거스로 변경
df2.loc[df2["prd"]=="아스파라가스","prd"] = "아스파라거스"
# 알 -> 알타리무로 변경
df2.loc[df2["prd"]=="알","prd"] = "알타리무"
# 알배 -> 알배기로 변경
df2.loc[df2["prd"]=="알배","prd"] = "알배기"
# 양채 -> 양채류로 통합
df2.loc[df2["prd"]=="양채","prd"] = "양채류"
# 얼갈이 -> 얼갈이배추로 통합
df2.loc[df2["prd"]=="얼갈이","prd"] = "얼갈이배추"
# 엽채류, 엽채 -> 엽경채류로 통합
df2.loc[df2["prd"]=="엽채류","prd"] = "엽경채류"
df2.loc[df2["prd"]=="엽채","prd"] = "엽경채류"
# 오징어 관련 품목들 정리
df2.loc[df2["prd"]=="오징어다리","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어원양","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어채","prd"] = "오징어"
df2.loc[df2["prd"]=="활복오징어","prd"] = "오징어"
# 옥, 풋옥 -> 옥수수로 정리
df2.loc[df2["prd"]=="옥","prd"] = "옥수수"
df2.loc[df2["prd"]=="풋옥","prd"] = "옥수수"
# 임연 -> 임연수어로 정리
df2.loc[df2["prd"]=="임연","prd"] = "임연수어"


# 절단 -> 절단쭈꾸미 -> 쭈꾸미로 정리
df2.loc[df2["prd"]=="절단","prd"] = "쭈꾸미"
# 조개살 -> 조개로 정리
df2.loc[df2["prd"]=="조개살","prd"] = "조개"

# 참조 -> 참조기로 정리
df2.loc[df2["prd"]=="참조","prd"] = "참조기"

# 칼라후라워,칼리플라워, 칼리후라워 -> 컬리플라워로 통합
df2.loc[df2["prd"]=="칼라후라워","prd"] = "컬리플라워"
df2.loc[df2["prd"]=="칼리플라워","prd"] = "컬리플라워"
df2.loc[df2["prd"]=="칼리후라워","prd"] = "컬리플라워"
# 코라비 -> 콜라비로 통합
df2.loc[df2["prd"]=="코라비","prd"] = "콜라비"
# 키조개날개 -> 키조개로 통합
df2.loc[df2["prd"]=="키조개날개","prd"] = "키조개"

# 팽이 -> 팽이버섯으로 통합
df2.loc[df2["prd"]=="팽이","prd"] = "팽이버섯"
# 파세리 -> 파슬리로 변경
df2.loc[df2["prd"]=="파세리","prd"] = "파슬리"
# 표고 -> 표고버섯으로 통합
df2.loc[df2["prd"]=="표고","prd"] = "표고버섯"
# 포장바지락 -> 바지락으로 변경
df2.loc[df2["prd"]=="포장바지락","prd"] = "바지락"

### 일단 여기까지 실행
# JY
#각굴 -> 굴로 통합
df2.loc[df2["prd"]=="각굴","prd"] = "굴"
# 게지 -> 키조개로 통합
df2.loc[df2["prd"]=="게지","prd"] = "키조개"
#고 -> 고구마로 통합
df2.loc[df2["prd"]=="고","prd"] = "고구마"
#꼴뚜 -> 꼴뚜기로 통합
df2.loc[df2["prd"]=="꼴뚜","prd"] = "꼴뚜기"
#돗나물 -> 돌나물로 통합
df2.loc[df2["prd"]=="돗나물","prd"] = "돌나물"
#레드쉬 -> 레디쉬로 통합
df2.loc[df2["prd"]=="레드쉬","prd"] = "레디쉬"
#만가닥 -> 만가닥버섯으로 통합
df2.loc[df2["prd"]=="만가닥","prd"] = "만가닥버섯"
#메 -> 메기로 통합
df2.loc[df2["prd"]=="메","prd"] = "메기"
#백조 -> 조기로 통합 
df2.loc[df2["prd"]=="백조","prd"] = "조기"
#봄동 -> 봄동배추로 통합
df2.loc[df2["prd"]=="봄동","prd"] = "봄동배추"
#봉지굴 -> 굴로 통합
df2.loc[df2["prd"]=="봉지굴","prd"] = "굴"
#붉은고추 -> 홍고추로 통합 
df2.loc[df2["prd"]=="붉은고추","prd"] = "홍고추"
#비 - > 비타민으로 통합
df2.loc[df2["prd"]=="비","prd"] = "비타민"
#빈스 -> 콩으로 통합
df2.loc[df2["prd"]=="빈스","prd"] = "콩"
#새송이 -> 새송이버섯으로 통합
df2.loc[df2["prd"]=="새송이","prd"] = "새송이버섯 "
#선동오징어 -> 오징어로 통합
df2.loc[df2["prd"]=="선동오징어","prd"] = "오징어"
#세트 -> 과채류로 통합
df2.loc[df2["prd"]=="세트","prd"] = "과채류"
#알 -> 알로애로 통합
df2.loc[df2["prd"]=="알","prd"] = "알로에"
#오징어 관련 품목들 정리
df2.loc[df2["prd"]=="선동오징어","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어몸통","prd"] = "오징어"
#오만둥이 -> 미더덕으로 통합
df2.loc[df2["prd"]=="오만둥이","prd"] = "미더덕"
#우렁쉥이 -> 멍게로 통합
df2.loc[df2["prd"]=="우렁쉥이","prd"] = "멍게"
#우뭇가시리 -> 우뭇가사리로 통합
df2.loc[df2["prd"]=="우뭇가시리","prd"] = "우뭇가사리"
#염고등어 -> 고등어로 통합
df2.loc[df2["prd"]=="염고등어","prd"] = "고등어"
#절단낙지 -> 낙지로 통합
df2.loc[df2["prd"]=="절단낙지","prd"] = "낙지"
#적채 -> 적양배추로 통합
df2.loc[df2["prd"]=="적채","prd"] = "적양배추"
#청어원양 -> 청어로 통합
df2.loc[df2["prd"]=="청어원양","prd"] = "청어"
#키조개꼭지 -> 키조개로 통합
df2.loc[df2["prd"]=="키조개꼭지","prd"] = "키조개"


In [294]:
# 첫 데이터기 때문에 products 파일 생성
# 기존 products 리스트와 비교
products = pd.DataFrame(df2['prd'].unique(),columns=['prd'])

In [295]:
products.to_csv('../products.csv')

In [266]:
# 전체 거래량과 품목량
# 비교를 위해 분포 파악
# 거래량이 너무 적은 품목은 삭제할 필요성이 있음

prd_lst = (df2['prd'].value_counts()).to_frame()
prd_lst.describe()

,prd
count,229.00000
mean,1768.61572
std,4813.67864
min,1.00000
25%,11.00000
50%,103.00000
75%,882.00000
max,43214.00000


In [267]:
# 50개 미만 품목은 비교가 힘드므로 삭제
# 줄어든 종류만 남기기

prd_lst = prd_lst[prd_lst['prd']>=50]
prd_lst.reset_index(inplace=True)
prd_lst = prd_lst.rename(columns = {'index':'prd','prd':'count'})

In [268]:
df3 = df2.merge(prd_lst)
df3[df3['count']>=50]['prd'].value_counts()

딸기      43214
버섯      30818
오이      17348
양채류     16642
감귤      16442
        ...  
미꾸라지       59
해삼         55
석류         54
김          53
곤달비        51
Name: prd, Length: 135, dtype: int64

In [269]:
# 줄어든 것 확인했으므로 count column은 삭제
df3 = df3.drop(columns='count')


### scale

In [270]:
np.set_printoptions(precision=2, suppress=True)
df3['scale'].unique()

array([    5.  ,     4.5 ,     2.  ,    10.  ,     1.5 ,     4.  ,
          16.5 ,     5.1 ,     5.4 ,     6.  ,    18.  ,     8.  ,
           5.01,     7.  ,    47.  ,    16.  ,     8.5 ,    20.  ,
          41.  ,     3.  ,    13.  ,     2.9 ,    50.  ,     3.8 ,
           3.9 ,     2.8 ,     4.1 ,     4.2 ,     4.3 ,     1.  ,
           3.1 ,    11.  ,     7.5 ,    14.  ,     0.5 ,     2.5 ,
           0.8 ,     9.  ,     9.6 ,     1.2 ,    15.  ,     2.4 ,
           5.45,    30.  ,     7.2 ,     2.6 ,     9.5 ,    13.5 ,
           6.5 ,    21.  ,     9.8 ,    12.  ,     9.7 ,     3.6 ,
           0.1 ,     3.3 ,     4.4 ,     6.7 ,    15.5 ,     6.75,
          11.5 ,     3.2 ,     5.5 ,     3.5 ,     2.3 ,     7.6 ,
          17.  ,     9.4 ,     0.2 ,     0.3 ,    12.5 ,     6.4 ,
           8.4 ,    11.4 ,     4.8 ,     2.7 ,     0.25,     0.13,
           0.05,     0.9 ,     0.6 ,     0.7 ,    11.2 ,    10.5 ,
           0.15,    24.  ,    22.  ,    23.  ,     0.4 ,     1

In [271]:
# 50kg 이상 거래된 건들 이상치라 판단하고 확인 진행
df3[df3['scale']>=50]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
5351,굴,50.00000,16000,경남 통영,normal,2021-01-16,3,320.00000
5358,굴,50.00000,15000,경남 통영,normal,2021-01-16,3,300.00000
137584,명태,215.00000,49000,수입 러시아,normal,2021-01-20,3,227.91000
137870,무,10000.00000,1650000,전북 고창군,normal,2021-01-04,1,165.00000
137871,무,10000.00000,1620000,전북 고창군,normal,2021-01-04,1,162.00000
137873,무,10000.00000,3380000,전북 고창군,normal,2021-01-04,1,338.00000
137874,무,10000.00000,1020000,전북 고창군,normal,2021-01-04,1,102.00000
137875,무,10000.00000,2750000,전북 고창군,normal,2021-01-04,1,275.00000
137876,무,10000.00000,4100000,전북 고창군,normal,2021-01-04,1,410.00000
137877,무,10000.00000,4460000,전북 고창군,normal,2021-01-04,1,446.00000


In [272]:
# df3[(df3['prd']=='양상추')&(df3['scale']>10)]
# df3[(df3['prd']=='배')]

In [273]:
# 스케일 오표기된 항목 수정
# 무는 kg 당 가격 확인 결과 실제 거래

# df2.loc[(df2['prd']=='당근') & (df2['scale']==200),'scale'] = 20

In [274]:
# 나머지는 이상치로 판단하고 모두 삭제
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']!='배')].index)
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']!='무')].index)

In [275]:
df3[df3['scale']>=50]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg


### origin

In [276]:
# 도,시, 국산, 수입 표기 분류 필요
df3['origin'].unique()

array(['경남 남해', '경남 마산(고성,진해)', '수입 중국', '경남 통영', '전남 여수', '경남 삼천포',
       '수입 러시아', '수입 미국', '전북 군산', '국내산', '경남 고성군', '경남 밀양시', '경남 진주시',
       '경남 창녕군', '경남 합천군', '경북 고령군', '경북 상주시', '대구 달성군', '경기 여주시',
       '경남 김해시', '경기 이천시', '경남 거제시', '경남 함안군', '전북 군산시', '광주시 광산구',
       '전북 임실군', '광주 광산구', '경기 여주군', '전남 나주시', '충남 논산시', '전북 익산시',
       '경북 성주군', '전북 전주시', '경기 안성시 일죽면', '경남 창원시', '전북 완주군', '경남 의령군',
       '경기 양평군', '충남 부여군', '전남 광양시', '경남 김해시 대동면', '경남', '광주시 남구',
       '경기 안성시 고삼면', '서울', '경남 밀양시 단장면', '경남 진주시 금곡면', '충남 서천군',
       '전남 고흥군 영남면', '경기 하남시', '전남 고흥(나로도)', '충남 서산', '수입 세네갈', '제주 제주도',
       '수입 기타', '수입남아프리카공화국', '제주 제주시', '제주', '제주 서귀포시', '제주 서귀포시(2)',
       '제주 북제주군', '제주 남제주군', '제주특별자치도 서귀', '제주 서귀포', '제주특별자', '제주 북제주',
       '제주도 서귀포시', '제주도 제주시', '전남 여수시', '경기 평택시', '전남 해남군', '충북 충주시',
       '경남 산청군', '전남 완도군', '경기 안성시', '전남 보성군', '경기 광주시', '전남 무안군',
       '전남 고흥군', '경북 청송군', '서울 중구', '충북 진천군', '전남 장흥군', '전남 장성군', '광주 북구',
       '경남 사천시', '전남 담양군'

In [277]:
# 해당 조건에 맞는 값 추출

splits = df3['origin'].str.split(' ')
cacul = splits.apply(lambda x: pd.Series(x))
df66 = pd.merge(df3,cacul, left_index=True, right_index=True, how='inner')
df66

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,0,1,2,3
0,가리비,5.00000,24000,경남 남해,normal,2021-01-04,3,4800.00000,경남,남해,NaN,NaN
1,가리비,5.00000,22000,"경남 마산(고성,진해)",normal,2021-01-04,3,4400.00000,경남,"마산(고성,진해)",NaN,NaN
2,가리비,5.00000,23000,경남 남해,normal,2021-01-04,3,4600.00000,경남,남해,NaN,NaN
3,가리비,5.00000,25000,"경남 마산(고성,진해)",normal,2021-01-04,3,5000.00000,경남,"마산(고성,진해)",NaN,NaN
4,가리비,5.00000,23500,"경남 마산(고성,진해)",normal,2021-01-04,3,4700.00000,경남,"마산(고성,진해)",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
403877,체리,5.00000,50000,칠레,normal,2021-01-19,2,10000.00000,칠레,NaN,NaN,NaN
403878,체리,5.00000,55000,칠레,normal,2021-01-21,2,11000.00000,칠레,NaN,NaN,NaN
403879,체리,5.00000,20000,칠레,normal,2021-01-04,9,4000.00000,칠레,NaN,NaN,NaN
403880,체리,5.00000,15000,칠레,normal,2021-01-09,9,3000.00000,칠레,NaN,NaN,NaN


In [278]:
# 세부 카테고리 정리 (state, city 편집 시 여기서 시작)

df4 = df66.rename(columns = {0:'state',1:'city'})

# 2,3은 필요없는 정보이므로 삭제
df4 = df4.drop(columns=[2,3])

In [279]:
df4['state'].unique()

array(['경남', '수입', '전남', '전북', '국내산', '경북', '대구', '경기', '광주시', '광주', '충남',
       '서울', '제주', '수입남아프리카공화국', '제주특별자치도', '제주특별자', '제주도', '충북', '대전',
       '강원', '인천', '강원도', '전주시', '세종', '전라북도', '인천시', '성남시', '중국', '포항시',
       '경기도', '울산', '서울특별시', '부산', '수입산', '충청남도', '필리핀', '전라남도', '경상남도',
       '미국', '서울시', '청주시', '세종시', '경북/문경시', '일본', '원양산', '부산시', '통가',
       '뉴칼레도니아', '뉴', '뉴질랜드', '누벨칼레도니', '인천광역시', '페루', '대전시', '칠레', '멕시코',
       '베트남', '이란', '러시아', '충북청주시상당구', '태국', '호주', '타이', '콜롬비아', '과테말라',
       '에콰도르', '인도네시아', '에쿠아도르', '에콰돌', '코스타리카', '이스라엘', '대만', '오스트레일리아',
       '아르헨티나', '기타외국', '남아프리카', '스페인', '남아프리카공화국'], dtype=object)

In [280]:
# 시,도 제거
def name_sort(words):
  p = re.compile('[^시도]+')
  word = re.search(p,str(words))
  return word.group()

df4["state"] = df4["state"].apply(name_sort)

# 오타도 제거

def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["state"] = df4["state"].apply(name_sort2)

In [281]:
df4["state"].unique()

array(['경남', '수입', '전남', '전북', '국내산', '경북', '대구', '경기', '광주', '충남', '서울',
       '제주', '수입남아프리카공화국', '제주특별자치', '제주특별자', '충북', '대전', '강원', '인천',
       '전주', '세종', '전라북', '성남', '중국', '포항', '울산', '서울특별', '부산', '수입산',
       '충청남', '필리핀', '전라남', '경상남', '미국', '청주', '경북/문경', '일본', '원양산', '통가',
       '뉴칼레', '뉴', '뉴질랜드', '누벨칼레', '인천광역', '페루', '칠레', '멕', '베트남', '이란',
       '러', '충북청주', '태국', '호주', '타이', '콜롬비아', '과테말라', '에콰', '인', '에쿠아',
       '에콰돌', '코스타리카', '이스라엘', '대만', '오스트레일리아', '아르헨티나', '기타외국', '남아프리카',
       '스페인', '남아프리카공화국'], dtype=object)

In [282]:
# state
# 특이케이스 제거

# 국내
df4.loc[df4["state"]=="국내산","city"] = "미상"
df4.loc[df4["state"]=="국내산","state"] = "국산"
df4.loc[df4["state"]=="원양산","city"] = "미상"

## 도단위
df4.loc[df4["state"]=="충청북","state"] = "충북"
df4.loc[df4["state"]=="충청남","state"] = "충남"
df4.loc[df4["state"]=="전라북","state"] = "전북"
df4.loc[df4["state"]=="전라남","state"] = "전남"
df4.loc[df4["state"]=="경상북","state"] = "경북"
df4.loc[df4["state"]=="경상남","state"] = "경남"

## 시단위
### 광역시

df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="대구","city"] = "대구"
df4.loc[df4["state"]=="서울특별","city"] = "서울"
df4.loc[df4["state"]=="서울특별","state"] = "서울"
df4.loc[df4["state"]=="인천광역","city"] = "인천"
df4.loc[df4["state"]=="인천광역","state"] = "인천"
df4.loc[df4["state"]=="부산","city"] = "부산"
df4.loc[df4["state"]=="가락동","city"] = "서울"
df4.loc[df4["state"]=="가락동","state"] = "서울"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="제주","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","state"] = "제주"
df4.loc[df4["state"]=="제주특별자","city"] = "제주"
df4.loc[df4["state"]=="제주특별자","state"] = "제주"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="울산","city"] = "울산"
df4.loc[df4["state"]=="울산","state"] = "울산"

### 일반시
df4.loc[df4["state"]=="성남","city"] = "성남"
df4.loc[df4["state"]=="성남","state"] = "경기"
df4.loc[df4["state"]=="전주","city"] = "전주"
df4.loc[df4["state"]=="전주","state"] = "전북"
df4.loc[df4["state"]=="청주","city"] = "청주"
df4.loc[df4["state"]=="청주","state"] = "충북"
df4.loc[df4["state"]=="수원","city"] = "수원"
df4.loc[df4["state"]=="수원","state"] = "경기"
df4.loc[df4["state"]=="부천","city"] = "부천"
df4.loc[df4["state"]=="부천","state"] = "경기"
df4.loc[df4["state"]=="경기고양","city"] = "고양"
df4.loc[df4["state"]=="경기고양","state"] = "경기"
df4.loc[df4["state"]=="충북청주","city"] = "청주"
df4.loc[df4["state"]=="충북청주","state"] = "충북"
df4.loc[df4["state"]=="전북전주","city"] = "전주"
df4.loc[df4["state"]=="전북전주","state"] = "전북"
df4.loc[df4["state"]=="경북/문경","city"] = "문경"
df4.loc[df4["state"]=="경북/문경","state"] = "경북"
df4.loc[df4["state"]=="포항","city"] = "포항"
df4.loc[df4["state"]=="포항","state"] = "경북"


# 외국
## 아시아
df4.loc[df4["state"]=="칠레","city"] = "칠레"
df4.loc[df4["state"]=="칠레","state"] = "수입"
df4.loc[df4["state"]=="일본","city"] = "일본"
df4.loc[df4["state"]=="일본","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="우즈베크","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베크","state"] = "수입"
df4.loc[df4["state"]=="이스라엘","city"] = "이스라엘"
df4.loc[df4["state"]=="이스라엘","state"] = "수입"
df4.loc[df4["state"]=="터키","city"] = "터키"
df4.loc[df4["state"]=="터키","state"] = "수입"
df4.loc[df4["state"]=="중국","city"] = "중국"
df4.loc[df4["state"]=="중국","state"] = "수입"
df4.loc[df4["state"]=="타이","city"] = "태국"
df4.loc[df4["state"]=="타이","state"] = "수입"
df4.loc[df4["state"]=="태국","city"] = "태국"
df4.loc[df4["state"]=="태국","state"] = "수입"
df4.loc[df4["state"]=="필리핀","city"] = "필리핀"
df4.loc[df4["state"]=="필리핀","state"] = "수입"
df4.loc[df4["state"]=="우즈베키스탄","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베키스탄","state"] = "수입"
df4.loc[df4["state"]=="대만","city"] = "대만"
df4.loc[df4["state"]=="대만","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="이란","city"] = "이란"
df4.loc[df4["state"]=="이란","state"] = "수입"
df4.loc[df4["state"]=="러","city"] = "러시아"
df4.loc[df4["state"]=="러","state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"city"] = "인도네시아"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"city"] = "인도"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"state"] = "수입"

## 아메리카

df4.loc[df4["state"]=="캘리포니아","city"] = "미국"
df4.loc[df4["state"]=="캘리포니아","state"] = "수입"
df4.loc[df4["state"]=="포클랜드","city"] = "영국"
df4.loc[df4["state"]=="포클랜드","state"] = "수입"
df4.loc[df4["state"]=="콜롬비아","city"] = "콜롬비아"
df4.loc[df4["state"]=="콜롬비아","state"] = "수입"
df4.loc[df4["state"]=="에콰","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰","state"] = "수입"
df4.loc[df4["state"]=="에쿠아","city"] = "에콰도르"
df4.loc[df4["state"]=="에쿠아","state"] = "수입"
df4.loc[df4["state"]=="에콰돌","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰돌","state"] = "수입"
df4.loc[df4["state"]=="과테말라","city"] = "과테말라"
df4.loc[df4["state"]=="과테말라","state"] = "수입"
df4.loc[df4["state"]=="코스타리카","city"] = "코스타리카"
df4.loc[df4["state"]=="코스타리카","state"] = "수입"
df4.loc[df4["state"]=="멕","city"] = "멕시코"
df4.loc[df4["state"]=="멕","state"] = "수입"
df4.loc[df4["state"]=="페루","city"] = "페루"
df4.loc[df4["state"]=="페루","state"] = "수입"
df4.loc[df4["state"]=="미국","city"] = "미국"
df4.loc[df4["state"]=="미국","state"] = "수입"
df4.loc[df4["state"]=="아르헨티나","city"] = "아르헨티나"
df4.loc[df4["state"]=="아르헨티나","state"] = "수입"

## 유럽
df4.loc[df4["state"]=="스페인","city"] = "스페인"
df4.loc[df4["state"]=="스페인","state"] = "수입"
df4.loc[df4["state"]=="우크라이나","city"] = "우크라이나"
df4.loc[df4["state"]=="우크라이나","state"] = "수입"

## 아프리카
df4.loc[df4["state"]=="남아프리카","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카","state"] = "수입"
df4.loc[df4["state"]=="남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카공화국","state"] = "수입"
df4.loc[df4["state"]=="수입남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="수입남아프리카공화국","state"] = "수입"


## 오세아니아
df4.loc[df4["state"]=="뉴질랜드","city"] = "뉴질랜드"
df4.loc[df4["state"]=="뉴질랜드","state"] = "수입"
df4.loc[df4["state"]=="오스트레일","city"] = "호주"
df4.loc[df4["state"]=="오스트레일","state"] = "수입"
df4.loc[df4["state"]=="오스트레일리아","city"] = "호주"
df4.loc[df4["state"]=="오스트레일리아","state"] = "수입"
df4.loc[df4["state"]=="호주","city"] = "호주"
df4.loc[df4["state"]=="호주","state"] = "수입"
df4.loc[df4["state"]=="통가","city"] = "통가"
df4.loc[df4["state"]=="통가","state"] = "수입"
df4.loc[df4["state"]=="뉴칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴칼레","state"] = "수입"
df4.loc[df4["state"]=="뉴","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴","state"] = "수입"
df4.loc[df4["state"]=="누벨칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="누벨칼레","state"] = "수입"
# 기타
df4.loc[df4["state"]=="수입산","state"] = "수입"
df4.loc[df4["state"]=="기타외국","city"] = "미상"
df4.loc[df4["state"]=="기타외국","state"] = "수입"

In [283]:
df4[df4['state']=='nan']

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,state,city


In [284]:
df4['state'].unique()

array(['경남', '수입', '전남', '전북', '국산', '경북', '대구', '경기', '광주', '충남', '서울',
       '제주', '충북', '대전', '강원', '인천', '세종', '울산', '부산', '원양산'],
      dtype=object)

In [285]:
# city
# 우선 결측치 제거
df4.loc[df4["city"].isnull(),"city"] = "미상"

In [286]:
# 이후 시,군 등 제거
def name_sort(words):
  p = re.compile('[^시군]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort)

In [287]:
# \t 도 제거
def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort2)

In [288]:
df4['city'].unique()

array(['남해', '마산(고성,진해)', '중국', '통영', '여수', '삼천포', '러', '미국', '산', '미상',
       '고성', '밀양', '진주', '창녕', '합천', '고령', '상주', '대구', '여주', '김해', '이천',
       '거제', '함안', '광산구', '임실', '나주', '논산', '익산', '성주', '전주', '안성', '창원',
       '완주', '의령', '양평', '부여', '광양', '남구', '서천', '고흥', '하남', '고흥(나로도)',
       '서산', '세네갈', '제주', '기타', '남아공', '평택', '해남', '충주', '산청', '완도', '보성',
       '광주', '무안', '청송', '중구', '진천', '장흥', '장성', '북구', '사천', '담양', '대전',
       '인제', '성남', '평창', '김제', '강릉', '예천', '부안', '홍천', '고창', '김천', '남원',
       '정선', '청양', '구미', '금산', '태안', '흥', '예산', '당진', '화천', '봉화', '원주',
       '철원', '춘천', '영주', '홍성', '제천', '부천', '안동', '횡성', '괴산', '세종', '신안',
       '강서구', '베트남', '바레인', '파키스탄', '장항(서천)', '장항', '강화', '영암', '아산',
       '천안', '보령', '음성', '송파구', '고양', '노르웨이', '하동', '마산', '공주', '서구',
       '청원', '강동구', '영동', '화순', '진도', '중랑구', '양주', '파주', '남양주', '도봉구',
       '포천', '동대문구', '용인', '구리', '양천구', '광명', '의왕', '경산', '영양', '의정부',
       '영광', '영천', '포항', '정읍', '영덕', '양구', '울산', '순천(광양)', '

In [289]:
# 나머지 케이스 정리

# 국내
df4.loc[df4["city"]=="기타","city"] = "미상"
## 일반시
df4.loc[df4["city"]=="위","city"] = "군위"
df4.loc[df4["city"]=="고흥(나로도)","city"] = "고흥"
df4.loc[df4["city"]=="산","city"] = "군산"
df4.loc[df4["city"]=="장항(서천)","city"] = "서천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="순천(광양)","city"] = "순천"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="화성(평택)","city"] = "화성"
df4.loc[df4["city"]=="흥","city"] = "시흥"
df4.loc[df4["city"]=="목포(무안)","city"] = "목포"
df4.loc[df4["city"]=="목포(무안)","state"] = "전남"
df4.loc[(df4["state"]=="전북")&(df4["city"]=="남"),"city"] = "남원"
## 광역시

df4.loc[df4["city"]=="강서구","city"] = "서울"
df4.loc[df4["city"]=="은평구","city"] = "서울"
df4.loc[df4["city"]=="강동구","city"] = "서울"
df4.loc[df4["city"]=="마포구","city"] = "서울"
df4.loc[df4["city"]=="중구","city"] = "서울"
df4.loc[df4["city"]=="송파구","city"] = "서울"
df4.loc[df4["city"]=="도봉구","city"] = "서울"
df4.loc[df4["city"]=="중랑구","city"] = "서울"
df4.loc[df4["city"]=="양천구","city"] = "서울"
df4.loc[df4["city"]=="성동구","city"] = "서울"
df4.loc[df4["city"]=="강남구","city"] = "서울"
df4.loc[df4["city"]=="동대문구","city"] = "서울"
df4.loc[df4["city"]=="영등포구","city"] = "서울"
df4.loc[df4["city"]=="영등포","city"] = "서울"
df4.loc[df4["city"]=="서초구","city"] = "서울"
df4.loc[df4["city"]=="구로구","city"] = "서울"
df4.loc[df4["city"]=="동대문","city"] = "서울"
df4.loc[df4["city"]=="금천구","city"] = "서울"

df4.loc[df4["city"]=="광산구","city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="남구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="북구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="서구"),"city"] = "광주"
df4.loc[df4["city"]=="동구","city"] = "광주"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="남구"),"city"] = "인천"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="서구"),"city"] = "인천"
df4.loc[df4["city"]=="미추홀","city"] = "인천"
df4.loc[df4["city"]=="부평구","city"] = "인천"
df4.loc[df4["city"]=="남동구","city"] = "인천"
df4.loc[df4["city"]=="미추홀구","city"] = "인천"
df4.loc[df4["city"]=="계양구","city"] = "인천"
df4.loc[df4["city"]=="마산(고성,진해)","city"] = "창원"
df4.loc[(df4["state"]=="부산")&(df4["city"]=="북구"),"city"] = "부산"
df4.loc[df4["city"]=="오포","city"] = "부산"
df4.loc[df4["city"]=="부산진","city"] = "부산"
df4.loc[df4["city"]=="사상구","city"] = "부산"
df4.loc[df4["city"]=="부산진구","city"] = "부산"
df4.loc[df4["city"]=="북제주","city"] = "제주"
df4.loc[df4["city"]=="남제주","city"] = "제주"

# 해외
## 아시아
df4.loc[df4["city"]=="러","city"] = "러시아"
df4.loc[df4["city"]=="말레이지아","city"] = "말레이시아"
df4.loc[df4["city"]=="타이","city"] = "태국"
df4.loc[df4["city"]=="인도네","city"] = "인도네시아"
## 아메리카
df4.loc[df4["city"]=="멕","city"] = "멕시코"
## 아프리카
df4.loc[df4["city"]=="남아공산","city"] = "남아공"
## 유럽
## 오세아니아

In [290]:
# 첫번째 파일이므로 cities 파일 생성
cities = pd.DataFrame(df4['city'].unique(),columns=['city'])

In [291]:
cities.to_csv('../cities.csv')

In [292]:
# 처리 완료하였으니 df4는 백업으로 두고 df5 활용
df5 = df4.drop(columns='origin')

# 전처리 결과

# 파일 저장

In [293]:
df5.to_csv('../data_proceed/202101_proceed.csv')